# Toronto Neighbourhoods - table
#### This is part of the Course [<u>*Applied Data Science Capstone*</u>](https://www.coursera.org/learn/applied-data-science-capstone/) on Coursera, to complete the Specialization <u>*IBM Data Science Professional Certificate*</u>

This exercise is to scraping a table from a [wikepedia page](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M). We're going to get Postcode, Borough and Neighbourhood values for Toronto and save in a pandas DataFrame, printing the number of rows at the end.

---------------
Importing dependencies:

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

Get the page and parse with BeatifulSoup:

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)

soup = BeautifulSoup(page.content,'html.parser')

The whole table is in `tbody` (not divided in `thead` and `tbody` like we would see in Chrome). However, headers are in the tag `th` and data in the tag `td`. We can get the whole table then with the CSS selector `table tbody`:

In [3]:
tablesoup = soup.select('table tbody')[0]

Table headers: 

In [4]:
columns = [x.get_text().replace('\n','') for x in tablesoup.select('tr')[0].select('th')]
columns

['Postcode', 'Borough', 'Neighbourhood']

Extract table content:

In [5]:
tb_content = [y.get_text().replace('\n','') for x in tablesoup.select('tr')[1:] for y in x.select('td')]
print('cells: %i' %len(tb_content))
print('rows: %.1f' %(len(tb_content)/3))

cells: 867
rows: 289.0


Extract columns and assign to a pandas DataFrame:

In [6]:
ttable = pd.DataFrame()
for i in range(3):
    ttable[columns[i]] = [tb_content[x] for x in np.arange(i,len(tb_content),3)]
ttable.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Remove Boroughs that have `Not assigned`

In [7]:
ttable_c1 = ttable[ttable.Borough != 'Not assigned'].reset_index(drop=True)
ttable_c1.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Assign Borough name to Neighbourhoods that are `Not assigned`

In [8]:
ttable_c1[ttable_c1.Neighbourhood == 'Not assigned']

,Postcode,Borough,Neighbourhood
6,M7A,Queen's Park,Not assigned


Only one of those cases, assigning just that one

In [9]:
ttable_c1.iat[6,2] = "Queen's Park"

In [10]:
ttable_c1.head(7)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park


Group by Postcode and join Neighbourhood names

In [11]:
ttable_c2 = ttable_c1.groupby(['Postcode','Borough'],
                              as_index=False,sort=False).agg(lambda x: ", ".join(x))

**Final table:**

In [12]:
ttable_c2.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


Save table to a csv (to use in another notebook): 

In [14]:
ttable_c2.to_csv('Toronto_Neighbourhoods.csv')

-----
Final objective, print the number of rows in the dataframe:

In [13]:
print('Number of rows in the dataframe: %i' %ttable_c2.shape[0])

Number of rows in the dataframe: 103
